# Counting experiment

- We observe a total number of $n = b+s$ events

- $s$ is the parameter of interest, $b$ is a nuisance parameter

- The likelihood $\mathcal{L}$ to observe $n$ events when the expected signal and background yields are $s$ and $b$ respectively is the Poissonian $\mathcal{P}$ with mean $s + b$ calculated at $n$

\begin{equation}
\mathcal{L}(s|b) = \mathcal{P}(n|s+b)
\end{equation}

- Place confidence interval both with a frequentist and bayesian technique on the number of signal events $s$

- Here we define some utils that will be useful for later calculation

#### Utils

In [ ]:
from scipy.stats import poisson,norm
from math import log
from scipy.integrate import simpson
from scipy.optimize import minimize
from functools import partial
from scipy.integrate import quad
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import cumulative_trapezoid

def model_poisson(n,s,b):
    return poisson(s+b).pmf(n)

def model_constraint(n,s,b,sigmab):
    return poisson(s+b).pmf(n)*norm(loc=3,scale=sigmab).pdf(b)

def model_poisson_2d(n,x):
    s = x[0]
    b = x[1]
    return poisson(s+b).pmf(n)
    
def model_constraint_2d(n,x,sigmab):
    s = x[0]
    b = x[1]
    return poisson(s+b).pmf(n)*norm(loc=3,scale=sigmab).pdf(b)

def nll(model,n,s,b,**kwargs):
    return -np.log(model(n,s,b,**kwargs))

def nll_2d(model,n,x,**kwargs):
    return -np.log(model(n=n,x=x,**kwargs))

def posterior(model,n,b=None,s=np.linspace(0,10,101),**kwargs):
    if b!=None: # fixed b
        integralmodel = quad(partial(model,n,b=b,**kwargs),0,np.inf)[0]
        return model(n,s,b,**kwargs)/integralmodel
    else: # b not fixed, optimise b
        post = []
        for s_i in s:
            res=minimize(partial(nll,model,n,s_i,**kwargs),x0=(3,),bounds=[(0,np.inf)])
            optimalb = res.x[0]
            #integralmodel = quad(partial(model,n,b=optimalb,**kwargs),0,np.inf)[0]
            post.append(model(n=n,s=s_i,b=optimalb,**kwargs))
        post = np.array(post)
        integralmodel = simpson(post,s)
        return post/integralmodel

def testStatistic(model,n,s,b,bestNLL,**kwargs):
    return 2*(nll(model=model,n=n,s=s,b=b,**kwargs)-bestNLL)

def testStatistic_2d(model,n,x,bestNLL,**kwargs):
    return 2*(nll_2d(model=model,n=n,x=x,**kwargs)-bestNLL)

## Bayesian approach

- The posterior is given by (Bayes theorem)

\begin{equation}
\mathrm{posterior}(s) = \frac{\mathcal{L}(s | b) \, \mathrm{prior}(s) }{\int \mathcal{L}(s | b) \, \mathrm{prior}(s) ds}
\end{equation}

- The prior for $s$ is uniform in the interval $s\geq 0$, and the posterior reduces to

\begin{equation}
\mathrm{posterior}(s) = \frac{\mathcal{L}(s | b) }{\int_0^\infty \mathcal{L}(s | b) ds}
\end{equation}

In the bayesian statistics, once you have the posterior you have “everything”, you can calculate intervals, upper limits, ...

### Bayesian interval

- In case the background $b$ is known the posterior is given by

\begin{equation}
\mathrm{posterior}(s) = \frac{\mathcal{L}(s | b)}{\int_0^\infty \mathcal{L}(s' | b) ds'} = \frac{\mathcal{P}(n|s+b)}{\int_o^\infty \mathcal{P}(n|s'+b) ds'} = \frac{\frac{(s+b)^n}{n!} e^{-(s+b)}}{\int_0^\infty \frac{(s'+b)^n}{n!} e^{-(s'+b)} ds'}
\end{equation}

- in the particular case $n=0$
\begin{equation}
\mathrm{posterior}(s) = \frac{e^{-(s+b)}}{\int_0^\infty e^{-(s'+b)} ds'} = e^{-s}
\end{equation}
the posterior (and then the confidence intervals) are indipendent on the number of expected background!

#### Python

In [ ]:
upper_limits = {}
fig,axs=plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True, figsize=(12,12))
s = np.linspace(0,10,10001)
for b in range(5):
    for n in range(5):
        post = posterior(model=model_poisson,n=n,b=b,s=s)
        cum_int = cumulative_trapezoid(post,s)
        CL = 0.9
        idx = (cum_int>CL).argmax()
        upper_limits[(n,b)] = s[idx]
        plt.sca(axs[b,n])
        plt.xlabel('s')
        plt.plot(s,post)

print('\t n=0 \t n=1 \t n=2 \t n=3 \t n=4')
for b in range(0,5):
    print('b='+str(b)+'\t'+( '\t'.join(['%.3f'%upper_limits[(n,b)] for n in range(0,5)]) ) )

#### ROOT - RooStats

In [ ]:
import ROOT

ws = ROOT.RooWorkspace()

ws.factory('expr::splusb(\'s+b\',s[1E-5,16],b[3,0,10])')
ws.factory('Poisson::poisson(n[0,100],splusb)')

ws.factory('Uniform::prior_s(s)') # prior signal
ws.factory('Gaussian::prior_b(b,3,sigmab[0.5,0.05,5.0])') # background with gaussian uncertainty
ws.factory('PROD::model(poisson,prior_b)')

modelc = ROOT.RooStats.ModelConfig(ws)
modelc = ROOT.RooStats.ModelConfig(ws)
modelc.SetPdf('poisson')
modelc.SetParametersOfInterest('s')
modelc.SetPriorPdf('prior_s')

c = ROOT.TCanvas('c','c',1000,800)
c.Divide(5,5,0.,0.)

upper_limits = {}
for b in range(0,5):
    #results.append([])
    for n in range(0,5):
        ws.var('b').setVal(b)
        ws.var('n').setVal(n)
        ws.var('b').setConstant()
        ws.var('n').setConstant()
        data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
        data.add(ROOT.RooArgSet(ws.var('n')))
        
        #obsn = ROOT.RooRealVar('obsn','obsn',n)
        #data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('x'),obsn),ROOT.RooFit.WeightVar('obsn'))
        #data.add(ROOT.RooArgSet(ws.var('x')),n)
        bcalc = ROOT.RooStats.BayesianCalculator(data,modelc)
        bcalc.SetLeftSideTailFraction(0.)
        bcalc.SetConfidenceLevel(0.9)
        interval = bcalc.GetInterval()
        upper_limits[(n,b)] = interval.UpperLimit()
        c.cd(b*5+n+1)
        frame = bcalc.GetPosteriorPlot()
        frame.Draw()

print('\t n=0 \t n=1 \t n=2 \t n=3 \t n=4')
for b in range(0,5):
    print('b='+str(b)+'\t'+( '\t'.join(['%.3f'%upper_limits[(n,b)] for n in range(0,5)]) ) )

ws.var('b').setConstant(False)
ws.var('n').setConstant(False)
c.Draw()

### Posterior with constraint

- To disentangle the parameter of interest $s$ with the nuisance parameter $b$, we add a constraint in $\mathcal{L}$ on $b$. It could be gaussian (in case previous measurements or estimates from high statistics control samples are available), or any other pdf that reflects the knowledge on this parameter
\begin{equation}
\mathcal{L}(s|b) = \mathcal{P}(n|s+b) \cdot \mathcal{G}(\overline{b},\sigma_b)
\end{equation}
where $\overline{b}$ is the estimate of background yield $b$ with an uncertainty $\sigma_b$

- For a given observation $n$, we obtain an optimised background yield $\hat{b}$, minimizing $-\log\mathcal{L}(s,b)$ for each $s$, such that

\begin{equation}
\mathrm{posterior}(s) = \frac{\mathcal{L}(s | \hat{b})}{\int_0^\infty \mathcal{L}(s' | \hat{b}) ds'} 
\end{equation}

#### Python

In [ ]:
for n in range(2,7):
    plt.plot(np.linspace(0,10,101),posterior(model=model_constraint,n=n,sigmab=0.5),label='n=%d'%n)
plt.legend(loc='best')
plt.xlabel('s')

#### ROOT - RooStats

In [ ]:
modelc.SetPdf('model')
modelc.SetParametersOfInterest('s')
modelc.SetPriorPdf('prior_s')
modelc.SetNuisanceParameters('b')

# bkg events not fixed (gaussian uncertainty)
frame = ws.var('s').frame()
legend = ROOT.TLegend(0.6,0.6,0.9,0.9)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
c = ROOT.TCanvas()
for i,n in enumerate(range(2,7)): # number of observed events
    ws.var('n').setVal(n)
    ws.var('n').setConstant()
    data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
    data.add(ROOT.RooArgSet(ws.var('n')))
    bcalc = ROOT.RooStats.BayesianCalculator(data,modelc)
    posterior = bcalc.GetPosteriorPdf()
    curve = posterior.plotOn(frame,ROOT.RooFit.LineColor(ROOT.kRed+i),ROOT.RooFit.LineWidth(3),ROOT.RooFit.Name('n%d'%n))
    legend.AddEntry(frame.findObject('n%d'%n),'n = %d'%n,'l')
ws.var('n').setConstant(False)
frame.Draw()
legend.Draw()
c.Draw()

### How the background knowledge impacts on the signal estimate

- The physical measurements profit of a better understanding and determination of the background

#### Python

In [ ]:
n=6
s = np.linspace(0,16,101)
for sigmab in [0.1,0.5,1.0,2.0]:
    post = posterior(model=model_constraint,n=6,s=s,sigmab = sigmab)
    plt.plot(s,post,label='$\sigma_b = %.1f$'%sigmab)
plt.legend(loc='best')
plt.xlabel('s')

#### ROOT - RooStats

In [ ]:
c = ROOT.TCanvas()
frame = ws.var('s').frame()
n=6
ws.var('n').setVal(n)
ws.var('n').setConstant()
ws.var('b').setVal(3)
ws.var('b').setConstant(False)

modelc = ROOT.RooStats.ModelConfig(ws)
modelc.SetPdf('model')
modelc.SetParametersOfInterest('s')
modelc.SetPriorPdf('prior_s')
modelc.SetNuisanceParameters('b')

data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
data.add(ROOT.RooArgSet(ws.var('n')))

legend = ROOT.TLegend(0.6,0.6,0.9,0.9)
legend.SetBorderSize(0)
legend.SetFillStyle(0)

for i,sigmab in enumerate([0.1,0.5,1.0,2.0]):
    ws.var('sigmab').setVal(sigmab)
    bcalc = ROOT.RooStats.BayesianCalculator(data,modelc)
    posterior = bcalc.GetPosteriorPdf()
    curve = posterior.plotOn(frame,ROOT.RooFit.LineColor(ROOT.kRed+i),ROOT.RooFit.LineWidth(3),ROOT.RooFit.Name('sigmab%.2f'%sigmab))
    legend.AddEntry(frame.findObject('sigmab%.2f'%sigmab),'#sigma_{b} = %.1f'%sigmab,'l')

frame.Draw()
legend.Draw()
c.Draw()

## Frequentist approach

- Define test statistic
\begin{equation}
t = − 2 \log \frac{\mathcal{L}(s , b )}{\mathcal{L}(\hat{s} , \hat{b} )}
\end{equation}
where $\hat{s}$, $\hat{b}$ identify the values for which we have the maximum $\mathcal{L}(s,b)$, obtained from the maximum likelihood fit

### Frequentist interval

- In the frequentist approach we use the Wilk’s theorem to convert the test statistic in significance (asymptotic limit) and then calculate intervals

- When the background is not constrained, the degree of freedom is $s + b$ and we can't disentangle the two contributions

- The best yields estimate is in the region where the test statistic is low and it is identified by $s + b=n=5$ (5 is the number of observed events)

#### Python

In [ ]:
n=5
res=minimize(partial(nll_2d,model_poisson_2d,n),x0=(0,1),bounds=[(0,np.inf),(0,np.inf),])
bestNLL = res.fun

b = np.linspace(1,6,101)
b = 0.5*(b[1:]+b[:-1])
s = np.linspace(1,6,101)
s = 0.5*(s[1:]+s[:-1])
sv, bv = np.meshgrid(s,b)

t = testStatistic_2d(model = model_poisson_2d,n=n, x = (sv,bv),bestNLL = bestNLL)
plt.hist2d(sv.flatten(),bv.flatten(),weights=t.flatten(),bins=[100,100])
plt.colorbar()
plt.xlabel('s')
plt.ylabel('b')

#### ROOT - RooStats

In [ ]:
import ROOT
n = 5
data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
ws.var('n').setVal(n)
data.add(ROOT.RooArgSet(ws.var('n')))

pdf = ws.pdf('poisson')

pl = ROOT.RooStats.ProfileLikelihoodCalculator(data,pdf,ROOT.RooArgSet(ws.var('s'),ws.var('b')))
interval = pl.GetInterval()

c = ROOT.TCanvas()
ROOT.SetOwnership(c,False)
intervalplot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
intervalplot.SetNPoints(100*100)
intervalplot.Draw('hist nominuit')
c.Draw()

### Test Statistic with constraint

- The likelihood $\mathcal{L}$ is modified as in the bayesian case
\begin{equation}
\mathcal{L}(s,b) = \mathcal{P}(n|s+b) \cdot \mathcal{G}(\overline{b},\sigma_b)
\end{equation}
and the test statistic $t$ is calculated accordingly

#### Python

In [ ]:
n=5
b = np.linspace(1,6,101)
b = 0.5*(b[1:]+b[:-1])
s = np.linspace(1,6,101)
s = 0.5*(s[1:]+s[:-1])
sv, bv = np.meshgrid(s,b)

#res=minimize(partial(nll_2d,partial(model_poisson_2d,n=n)),x0=(0,1),bounds=[(0,np.inf),(0,np.inf),])
#print(minimize(partial(nll_2d,model=partial(model_poisson_2d,n=n,sigmab=sigmab)),x0=(0,1),bounds=[(0,np.inf),(0,np.inf),]))

from scipy.stats import chi2
sigma1 = norm.cdf(-1)*2
sigma2 = norm.cdf(-2)*2
sigma3 = norm.cdf(-3)*2

for sigmab in [0.25,1.,1.25,1.5,2.0]:
    res=minimize(partial(nll_2d,model_constraint_2d,n,sigmab=sigmab),x0=(0,1),bounds=[(0,np.inf),(0,np.inf),])
    bestNLL = res.fun
    t = testStatistic_2d(model=model_constraint_2d, n=n,x = (sv,bv),bestNLL = bestNLL,sigmab=sigmab)
    plt.figure()
    plt.title(r'$\sigma_b=%.2f$'%sigmab)
    plt.hist2d(sv.flatten(),bv.flatten(),weights=t.flatten(),bins=[100,100])
    plt.colorbar()
    plt.contour(s,b,t,[chi2(2).ppf(1-sigma1),chi2(2).ppf(1-sigma2),chi2(2).ppf(1-sigma3)],colors=['red','tomato','pink'])
    plt.scatter(res.x[0],res.x[1])
    plt.xlabel('s')
    plt.ylabel('b')

#### ROOT - RooStats

In [ ]:
n = 5
data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
ws.var('n').setVal(n)
data.add(ROOT.RooArgSet(ws.var('n')))

for sigmab in [0.25,1,1.25,1.5,2.0]:
    ws.var('sigmab').setVal(sigmab)
    ws.var('sigmab').setConstant()
    pl = ROOT.RooStats.ProfileLikelihoodCalculator(data,ws.pdf('model'),ROOT.RooArgSet(ws.var('s'),ws.var('b')))    
    interval = pl.GetInterval()
    c = ROOT.TCanvas('c'+str(sigmab),'c'+str(sigmab))
    ROOT.SetOwnership(c,False)
    intervalplot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
    intervalplot.SetTitle('#sigma_{b} = %.2f'%sigmab)
    intervalplot.SetNPoints(100*100)
    intervalplot.Draw('hist nominuit')
    #h = intervalplot.GetPlottedObject()
    #h.Draw('colz2')
    #h.SetTitle('#sigma_{b} = %.2f'%sigmab)
    #h.Draw('colz2')
    #c.SaveAs('LogL_b_%.2f.pdf'%sigmab)
    c.Draw()
ws.var('sigmab').setConstant(False)

### Profiled Likelihood ratio

- The number of background yield $b$ is considered as nuisance parameter and is profiled in the test statistic, i.e. for each value of s an optimal value of $\hat{\hat{b}}$ is obtained minimazing the negative log likelihood
\begin{equation}
t = − 2 \log \frac{\mathcal{L}(s , \hat{\hat{b}} )}{\mathcal{L}(\hat{s} , \hat{b} )}
\end{equation}

#### Python

In [ ]:
n=5
CL = 0.9
limit = np.sqrt(chi2(1).ppf(CL))
s = np.linspace(0.5,8,151)
for sigmab in [0.25,1.,1.25,1.5,2.0]:
    res = minimize(partial(nll_2d,model_constraint_2d,n,sigmab=sigmab),x0=(2,3),bounds=[(0,np.inf),(0,np.inf),])
    bestNLL = res.fun
    t = np.array([])
    for si in s:
        res = minimize(partial(nll,model_constraint,n,si,sigmab=sigmab),x0=(3,),bounds=[(0,np.inf),])
        #bestNLL = res.fun[0]
        #print(bestNLL)
        t = np.append(t,testStatistic(model=model_constraint,n=n,s=si,b=res.x[0],bestNLL=bestNLL,sigmab=sigmab))
        #t.append(testStatistic(model_constraint,si,b=res.x[0],bestNLL))
    p = plt.plot(s,t,label=r'$\sigma_b=%.2f$'%sigmab)
    idx = (t>limit).argmax()
    upper_limit = 0.5*(s[idx]+s[idx-1])
    print('sigmab = %.2f, upper limit = %.2f'%(sigmab,upper_limit))
    plt.vlines(upper_limit,0,limit,lw=1,color=p[0].get_color())
plt.xlabel('s')
plt.axhline(limit,color='black',lw=1)
plt.legend(loc='best')

#### ROOT - RooStats

In [ ]:
n = 5
data = ROOT.RooDataSet('data','data',ROOT.RooArgSet(ws.var('n')))
ws.var('n').setVal(n)
data.add(ROOT.RooArgSet(ws.var('n')))

c = ROOT.TCanvas()
ROOT.SetOwnership(c,False)
colors = [ROOT.kBlack,ROOT.kOrange,ROOT.kBlue,ROOT.kRed,ROOT.kMagenta]
legend = ROOT.TLegend(0.5,0.6,0.9,0.9)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
for i,sigmab in enumerate([0.25,1,1.25,1.5,2.0]):
    ws.var('sigmab').setVal(sigmab)
    ws.var('sigmab').setConstant()
    pl_s = ROOT.RooStats.ProfileLikelihoodCalculator(data,ws.pdf('model'),ROOT.RooArgSet(ws.var('s')))
    interval_s = pl_s.GetInterval()
    interval_s.SetConfidenceLevel(0.8) # 90% one sided for upper limit
    print('sigma_b = %.2f, upper limit on s at 90%%CL = %.2f'%(sigmab,interval_s.UpperLimit(ws.var('s'))))
    intervalplot_s = ROOT.RooStats.LikelihoodIntervalPlot(interval_s)
    intervalplot_s.SetMaximum(1.5)
    intervalplot_s.SetNPoints(100)
    intervalplot_s.SetLineColor(colors[i])
    intervalplot_s.SetContourColor(colors[i])
    intervalplot_s.Draw('same' if i!=0 else '')
    intervalplot_s.Print('v')
    h = intervalplot_s.GetPlottedObject()
    h.Print('v')
    #print(h.ls())
    #print(dir(h))
    #h.getCurve('nll_model_poissonData_with_constr_Profile[s]_Norm[s]').SetLineColor(colors[i])
    #h.Draw('same' if i!=0 else '')
    #legend.AddEntry(h.getCurve(0),'#sigma_{b} = %.2f'%sigmab,'l')
    legend.AddEntry(h.getCurve('nll_model_data_with_constr_Profile[s]_Norm[s]'),'#sigma_{b} = %.2f'%sigmab,'l')

legend.Draw()
c.Draw()